<a href="https://colab.research.google.com/github/tsangrebecca/BloomTech/blob/main/Sprint14/M3_DS_423_Tune_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

# Gridsearch Hyperparameters

In the guided project, you learned how to use sklearn's `GridsearchCV` and `keras-tuner` libraries to tune the hyperparameters of a neural network model. For your module project you'll continue using these two libraries, however we are going to make things a little more interesting for you.

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class.



**Don't forget to switch to GPU on Colab!**

## 0.1 Imports and installs

In [1]:
pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [1]:
pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [1]:
# native python libraries imports
import math
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn imports
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# keras imports
from keras import Sequential
from keras.layers import Dense
from keras.activations import relu, sigmoid
from keras.optimizers import Adam, SGD
from keras.utils import get_file

# required for compatibility between sklearn and keras
from scikeras.wrappers import KerasClassifier, KerasRegressor

# install keras-tuner
from keras_tuner import RandomSearch, BayesianOptimization
from keras_tuner import HyperParameters

## 0.2 Load quickdraw data

In [2]:
def load_quickdraw10():
    """
    Fill out this doc string, and comment the code, for practice in writing the kind of code that will get you hired.
    """

    URL_ = "https://github.com/LambdaSchool/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/quickdraw10.npz?raw=true"

    path_to_zip = get_file('./quickdraw10.npz', origin=URL_, extract=False)

    data = np.load(path_to_zip)

    # normalize your image data
    max_pixel_value = 255
    X = data['arr_0']/max_pixel_value
    Y = data['arr_1']

    return train_test_split(X, Y, shuffle=True)

In [3]:
X_train, X_test, y_train, y_test = load_quickdraw10()

25421363/25421363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
X_train.shape

(75000, 784)

In [5]:
y_train.shape

(75000,)

_____

# Experiment 1

## Tune Hyperperameters using Enhanced GridsearchCV

Enhanced by automation instead of manually having to do it ourselves.

We are going to use GridsearchCV again to tune a deep learning model however we are going to add some additional functionality to our gridsearch.

Specifically, we are going to automate the generation of how many nodes to use in a layer and how many layers to use in a model!

By the way, yes, there is a function within a function. Try to not let that bother you. An alternative to this would be to create a class. If you're up for the challenge give it a shot. However, consider this a stretch goal that you come back to after you finish going through this assignment.


### Objective

The objective of this experiment is to show you how to automate the generation of layers and layer nodes for the purposes of gridsearch. <br>
Up until now, we've been manually selecting the number of layers and layer nodes.

In [6]:
# Function to create model, required for KerasClassifier
def create_model(n_layers,  first_layer_nodes, last_layer_nodes, act_funct ="relu", negative_node_incrementation=True):
    """"
    Returns a compiled keras model

    Parameters
    ----------
    n_layers: int
        number of hidden layers in model
        To be clear, this excludes the input and output layer.

    first_layer_nodes: int
        Number of nodes in the first hidden layer

    last_layer_nodes: int
        Number of nodes in the last hidden layer (this is the layer just prior to the output layer)

     act_funct: string
         Name of activation function to use in hidden layers (this excludes the output layer)

    Returns
    -------
    model: compiled keras model
    """
    # Nested function
    def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
        """
        Generates and returns the number of nodes in each hidden layer.
        To be clear, this excludes the input and output layer.

        Note
        ----
        Number of nodes in each layer is linearly incremented.
        For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

        Parameters
        ----------
        n_layers: int
            Number of hidden layers
            This values should be 2 or greater

        first_layer_nodes: int

        last_layer_nodes: int

        Returns
        -------
        layers: list of ints
            Contains number of nodes for each layer
        """

        # throws an error if n_layers is less than 2
        assert n_layers >= 2, "n_layers must be 2 or greater"

        # Save the number of nodes for each layer here
        layers = []

        # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
        #   THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
        # when set to True number of nodes are decreased for subsequent layers
        # NOTE: the order of the number of nodes doesn't matter
        # The incrementation is linear, spacing from layer to layer is equal

        if negative_node_incrementation:
            # subtract this amount from previous layer's nodes in order to decrease towards smaller numbers e.g. from 500 to 400, 300, 200, etc.
            nodes_increment = (last_layer_nodes - first_layer_nodes) / (n_layers - 1) # e.g. 100 - 500 is a negative number, so it's decreasing

        # when set to False number of nodes are increased for subsequent layers, opposite to above
        else:
            # add this amount from previous layer's nodes in order to increment towards larger numbers, just use addition instead of subtraction?
            nodes_increment = (first_layer_nodes - last_layer_nodes)/ (n_layers - 1)
            # This follows the solution video code, e.g. 500 - 100 is a positive number so it's increasing increment
            # original notebook has it as nodes_increment = (last_layer_nodes + first_layer_nodes)/ (n_layers-1) ????

        nodes = first_layer_nodes

        for i in range(1, n_layers+1):

            layers.append(math.ceil(nodes))

            # increment nodes for next layer
            nodes = nodes + nodes_increment

        return layers

    # create model
    # Instantiate the Sequential class and call it model
    model = Sequential()

    # Pass values to the function above called gen_layer_nodes
    n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)

    # Create all hidden layers here using .add()
    for i in range(1, n_layers+1):
        # First layer, need to provide input data
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=X_train.shape[1], activation=act_funct))
        # Add additional layers
        else:
            model.add(Dense(n_nodes[i-1], activation=act_funct))


    # output layer
    model.add(Dense(10, # 10 unit/neurons in output layer because we have 10 possible labels to predict
                    activation='softmax')) # use softmax for a label set greater than 2

    # Compile model
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam', # adam is a good default optimizer
                  metrics=['accuracy'])

    # do not include model.fit() inside the create_model function
    # KerasClassifier is expecting a compiled model
    return model


## 1.1 Explore `create_model`



The helper function `gen_layer_nodes()` which is contained inside `create_model()` <br>
returns a list containing the number of nodes for each successive layer.<br>

Let's check that `gen_layer_nodes()` behaves as expected. <br>
In other words, we'll perform a **Unit Test!**

In [7]:
def gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation=True):
    """
    Generates and returns the number of nodes in each hidden layer.
    To be clear, this excludes the input and output layer.

    Note
    ----
    Number of nodes in each layer is linearly incremented.
    For example, gen_layer_nodes(5, 500, 100) will generate [500, 400, 300, 200, 100]

    Parameters
    ----------
    n_layers: int
        Number of hidden layers
        This values should be 2 or greater

    first_layer_nodes: int

    last_layer_nodes: int

    Returns
    -------
    layers: list of ints
        Contains number of nodes for each layer
    """

    # throws an error if n_layers is less than 2
    assert n_layers >= 2, "n_layers must be 2 or greater"

    layers = []

    # PROTIP: IF YOU WANT THE NODE INCREMENTATION TO BE SPACED DIFFERENTLY
    # THEN YOU'LL NEED TO CHANGE THE WAY THAT IT'S CALCULATED - HAVE FUN!
    # when set to True number of nodes are decreased for subsequent layers
    # NOTE: the order of the number of nodes doesn't matter
    if negative_node_incrementation:
        # subtract this amount from previous layer's nodes in order to increment towards smaller numbers
        nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
        #print(f'nodes increment = {nodes_increment}')

    # when set to False number of nodes are increased for subsequent layers
    else:
        # add this amount from previous layer's nodes in order to increment towards larger numbers
        nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
        #print(f'nodes increment = {nodes_increment}')

    nodes = first_layer_nodes

    for i in range(1, n_layers+1):

        layers.append(math.ceil(nodes))

        # increment nodes for next layer
        nodes = nodes + nodes_increment

    return layers


### `negative_node_incrementation = True`
For this case we want the number of nodes to _decrease_ by a constant number for successive layers. <br>So `first_layer_nodes` must be _larger_ than `last_layer_nodes`

In [8]:
n_layers = 5
first_layer_nodes = 500
last_layer_nodes = 100
negative_node_incrementation = True
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [500, 400, 300, 200, 100]


### `negative_node_incrementation = False`
For this case we want the number of nodes to _increase_ by a constant number for successive layers. <br>So `first_layer_nodes` must be _smaller_ than `last_layer_nodes`

In [9]:
n_layers = 5
first_layer_nodes = 100
last_layer_nodes = 500
negative_node_incrementation = False
n_nodes = gen_layer_nodes(n_layers, first_layer_nodes, last_layer_nodes, negative_node_incrementation)
print(f'Number of nodes in successive layers: {n_nodes}')

Number of nodes in successive layers: [100, 200, 300, 400, 500]


### OK, the Unit Test is passed!

### Let's build a few models<br>
in order to understand how `create_model()` works in practice.

### Build a model, setting `negative_node_incrementation = True`

Use `create_model` to build a model.

- Set `n_layers = 10`
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`


In [10]:
# use create_model to create a model

# YOUR CODE HERE
model = create_model(n_layers = 10,
                     first_layer_nodes = 500,
                     last_layer_nodes = 100,
                     act_funct = 'relu',
                     negative_node_incrementation=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# run model.summary() and make sure that you understand the model architecture that you just built
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values.
# Decreasing number of nodes
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 500)                 │         392,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 456)                 │         228,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 412)                 │         188,284 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 367)                 │         151,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 323)                 │         118,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 278)                 │          90,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 234)                 │          65,286 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 189)                 │          44,415 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 145)                 │          27,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 100)                 │          14,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,322,608 (5.05 MB)

 Trainable params: 1,322,608 (5.05 MB)

 Non-trainable params: 0 (0.00 B)

### Build a model, setting `negative_node_incrementation = False`

Use `create_model` to build a model.

- Set `n_layers = 10`
- Set `first_layer_nodes = 100`
- Set `last_layer_nodes = 500`
- Set `act_funct = "relu"`

In [24]:
# use create_model to create a model

# YOUR CODE HERE
model = create_model(n_layers = 10,
                     first_layer_nodes = 500,
                     last_layer_nodes = 100,
                     act_funct = 'relu',
                     negative_node_incrementation=False)

In [13]:
# run model.summary() and make sure that you understand the model architecture that you just built
# Notice in the model summary how the number of nodes have been linearly incremented in decreasing values
# Increasing number of nodes
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                     │ (None, 500)                 │         392,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 545)                 │         273,045 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 589)                 │         321,594 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 634)                 │         374,060 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 678)                 │         430,530 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 723)                 │         490,917 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 767)                 │         555,308 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 812)                 │         623,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 856)                 │         695,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 901)                 │         772,157 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 10)                  │           9,020 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,938,675 (18.84 MB)

 Trainable params: 4,938,675 (18.84 MB)

 Non-trainable params: 0 (0.00 B)

## 1.2 Create a grid search using `sklearn`

### Hyperparameter search

In [14]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3],
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [21]:
model = KerasClassifier(create_model, first_layer_nodes=500, last_layer_nodes=100, n_layers=2)

In [22]:
%%time
# Create Grid Search
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-2,
                    verbose=1,
                    cv=3)

grid_result = grid.fit(X_train, y_train) # we had to split the data so we're only training on 66% of the data which could explain why the accuracy score is lower than other models

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.7491 - loss: 0.8127
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 10ms/step - accuracy: 0.8642 - loss: 0.4438
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.8889 - loss: 0.3559
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.7466 - loss: 0.8188
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.8654 - loss: 0.4399
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.8917 - loss: 0.3532
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


...

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7325 - loss: 0.8609
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.8578 - loss: 0.4638
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8847 - loss: 0.3738
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7331 - loss: 0.8578
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8583 - loss: 0.4646
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8840 - loss: 0.3731
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.7689 - loss: 0.7525
Epoch 2/3
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 53s 14ms/step - accuracy: 0.8750 - loss: 0.4129
Epoch 3/3
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - accuracy: 0.8990 - loss: 0.3342
Best: 0.8659333333333334 using {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8659333333333334, Stdev: 0.0036354305873659183 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8624666666666667, Stdev: 0.0026179550968053115 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 3}
Means: 0.8614666666666667, Stdev: 0.005611543657695477 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 2}
Means: 0.8623066666666667, Stdev: 0.0033053223073636145 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 3}
Means: 0.8616933333333333, Stdev: 0.0027655660461387475 wit

In [25]:
best_model = grid_result.best_estimator_

In [26]:
best_model.get_params()

{'model': <function __main__.create_model(n_layers, first_layer_nodes, last_layer_nodes, act_funct='relu', negative_node_incrementation=True)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': 'rmsprop',
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 100,
 'n_layers': 2,
 'class_weight': None}

Ok, now that we've played around a bit with  `create_model`, let's build a  simpler model that we'll use to run gridsearches.

### Build model

Use `create_model` to build a model.

- Set `n_layers = 2`
- Set `first_layer_nodes = 500`
- Set `last_layer_nodes = 100`
- Set `act_funct = "relu"`
- Make sure that `negative_node_incrementation = True`

In [28]:
# use create_model to create a model

###BEGIN SOLUTION
# use create_model to create a model
model = create_model(n_layers = 2,
                     first_layer_nodes = 500,
                     last_layer_nodes = 100,
                     act_funct = 'relu',
                     negative_node_incrementation=True)
###END SOLUTION

In [29]:
# run model.summary() and make sure that you understand the model architecture that you just built
model.summary()

Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_124 (Dense)                    │ (None, 500)                 │         392,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_125 (Dense)                    │ (None, 100)                 │          50,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_126 (Dense)                    │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 443,610 (1.69 MB)

 Trainable params: 443,610 (1.69 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# define the grid search parameters
param_grid = {'n_layers': [2, 3],
              'epochs': [3],
              "first_layer_nodes": [500, 300],
              "last_layer_nodes": [100, 50]
             }

In [39]:
model = KerasClassifier(create_model, first_layer_nodes=500, last_layer_nodes=100, n_layers=2)

In [40]:
%%time
# Create Grid Search
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    n_jobs=-2,
                    verbose=1,
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.7388 - loss: 0.8391
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.8649 - loss: 0.4439
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8939 - loss: 0.3500
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.7497 - loss: 0.8083
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.8662 - loss: 0.4400
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8931 - loss: 0.3481
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


...

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7386 - loss: 0.8466
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.8583 - loss: 0.4601
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8867 - loss: 0.3689
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.7236 - loss: 0.8920
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 24s 9ms/step - accuracy: 0.8551 - loss: 0.4785
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.8851 - loss: 0.3736
782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.7654 - loss: 0.7591
Epoch 2/3
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.8729 - loss: 0.4207
Epoch 3/3
2344/2344 ━━━━━━━━━━━━━━━━━━━━ 41s 9ms/step - accuracy: 0.8961 - loss: 0.3398
Best: 0.8650666666666668 using {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8650666666666668, Stdev: 0.0019111834611628077 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 2}
Means: 0.8614133333333333, Stdev: 0.003178986561084497 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 100, 'n_layers': 3}
Means: 0.86316, Stdev: 0.001883259585576719 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 2}
Means: 0.8560533333333332, Stdev: 0.005078617484666004 with: {'epochs': 3, 'first_layer_nodes': 500, 'last_layer_nodes': 50, 'n_layers': 3}
Means: 0.86088, Stdev: 0.000650743164901992 with: {'epochs': 3, 'first_la

In [41]:
best_model = grid_result.best_estimator_

In [42]:
best_model.get_params()

{'model': <function __main__.create_model(n_layers, first_layer_nodes, last_layer_nodes, act_funct='relu', negative_node_incrementation=True)>,
 'build_fn': None,
 'warm_start': False,
 'random_state': None,
 'optimizer': 'rmsprop',
 'loss': None,
 'metrics': None,
 'batch_size': None,
 'validation_batch_size': None,
 'verbose': 1,
 'callbacks': None,
 'validation_split': 0.0,
 'shuffle': True,
 'run_eagerly': False,
 'epochs': 3,
 'first_layer_nodes': 500,
 'last_layer_nodes': 100,
 'n_layers': 2,
 'class_weight': None}

-----

# Experiment 2: Run the Gridsearch Algorithms

In this section, we are going to use the same model and dataset in order to benchmark 3 different gridsearch approaches:

- Gridsearch
- Random Search
- Bayesian Optimization.


Our goal in this experiment is two-fold. We want to see which appraoch

- Scores the highest accuracy
- Has the shortest run time

We want to see how these 3 gridsearch approaches handle these trade-offs and to give you a sense of those trades offs.

### Trade-offs

`Gridsearch` will train a model on every single unique hyperparameter combination, this guarantees that you'll get the highest possible accuracy from your parameter set but your gridsearch might have a very long run-time.

`Random Search` will randomly sample from your parameter set which, depending on how many samples, the run-time might be significantly cut down but you might or might not sample the parameters that correspond to the heightest possible accuracies.

`Bayesian Optimization` has a bit of intelligence built into it's search algorithm but you do need to manually select some parameters which may greatly influence the model learning outcomes.

-------
### Build our hyperparameter dictionary

In [43]:
# build out our hyperparameter dictionary
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3])
hp.Choice('activation', values=["relu", "sigmoid"])

'relu'

---------
## 2.1 Gridsearch Optimization


###Brute Force Gridsearch Optimization (in the last part of solution video)

### Populate a `sklearn` compatible parameter dictionary

In [71]:
# build out our hyperparameter dictionary
hyper_parameters = {
    # BUG Fix: cast array as list otherwise GridSearchCV will throw error
    "units": np.arange(32, 544, 32).tolist(), # you want the units to be 32 to 512, so the range has to go from 32 to 544
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "activation":["relu", "sigmoid"]
}

In [72]:
hyper_parameters

{'units': [32,
  64,
  96,
  128,
  160,
  192,
  224,
  256,
  288,
  320,
  352,
  384,
  416,
  448,
  480,
  512],
 'learning_rate': [0.1, 0.01, 0.001],
 'activation': ['relu', 'sigmoid']}

### Build a `sklearn` compatible model function

In [73]:
def build_model(units, learning_rate, activation):
    """
    Returns a complie keras model ready for keras-tuner gridsearch algorithms
    """

    model = Sequential()

    # hidden layer
    model.add(Dense(units, activation=activation))

    # output layer
    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

### Apply the "wrapper" to make the model compatible with `sklearn`

In [74]:
model = KerasClassifier(model=build_model, activation=relu, learning_rate=0.1, units=32)

In [75]:
# save start time
start = time()

# Create Grid Search
grid = GridSearchCV(estimator=model,
                    param_grid=hyper_parameters,
                    n_jobs=-2,
                    verbose=1,
                    cv=3)

grid_result = grid.fit(X_train, y_train)

# save end time
end = time()

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 96 candidates, totalling 288 fits


ValueError: 
All the 288 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 770, in fit
    self._fit(
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 936, in _fit
    self._check_model_compatibility(y)
  File "/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py", line 559, in _check_mode

In [ ]:
# total run time
total_run_time_in_minutes = (end - start)/60
total_run_time_in_minutes

In [ ]:
grid_result.best_params_

In [ ]:
# because all other optimization approaches are reporting test set score
# let's calculate the test set score in this case
best_model = grid_result.best_estimator_
test_acc = best_model.score(X_test, y_test)

In [ ]:
test_acc

 ### Results

Identify and write the the best performing hyperparameter combination and model score.



YOUR ANSWER HERE

------
## 2.2 Random Search with `keras-tuner`

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `RandomSearch` tuner.

### Build model
---

In [57]:
# because gridsearching can take a lot of time and we are bench marking 3 different approaches
# let's build a simple model to minimize run time

def build_model(hp):
    """
    Returns a complied keras model ready for keras-tuner gridsearch algorithms
    """

    model = Sequential()

    # hidden layer
    # Passing the object called hp, calling the .get attribute, and specify the names of the parameters
    model.add(Dense(units=hp.get('units'),activation=hp.get("activation")))

    # output layer
    model.add(Dense(10, activation='softmax'))

    model.compile(
        optimizer=Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

In [58]:
# how many unique hyperparameter combinations do we have?
# HINT: take the product of the number of possible values for each hyperparameter
# save your answer to n_unique_hparam_combos

# YOUR CODE HERE
n_values_in_units = 512/32 # Divide max and the min --> 16
n_values_in_learning_rate = 3
n_values_in_activation = 2

n_unique_hparam_combos = n_values_in_units * n_values_in_learning_rate * n_values_in_activation
n_unique_hparam_combos


96.0

In [59]:
# how many of these do we want to randomly sample?
# let's pick 25% of n_unique_hparam_combos param combos to sample
# save this number to n_param_combos_to_sample

# YOUR CODE HERE
fraction_to_sample = 0.25
n_param_combos_to_sample = int(n_unique_hparam_combos * fraction_to_sample)
n_param_combos_to_sample

24

### Instantiate a `RandomSearch()` object for your grid search

In [60]:
random_tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=n_param_combos_to_sample, # number of times to sample the parameter set and build a model
    seed=1234,
    hyperparameters=hp, # pass in our hyperparameter dictionary
    directory='./keras-tuner-trial',
    project_name='random_search'
)

In [61]:
 # take note of Total elapsed time in print out -- took ~10 minutes without GPU
random_tuner.search(
    X_train,
    y_train,
    epochs=3,
    validation_data=(X_test, y_test),
)

Trial 24 Complete [00h 01m 00s]
val_accuracy: 0.6713600158691406

Best val_accuracy So Far: 0.8747599720954895
Total elapsed time: 00h 25m 44s


In [62]:
# identify the best score and hyperparamter (should be at the top since scores are ranked)
random_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial/random_search
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 13 summary
Hyperparameters:
units: 384
learning_rate: 0.001
activation: relu
Score: 0.8747599720954895

Trial 18 summary
Hyperparameters:
units: 256
learning_rate: 0.001
activation: relu
Score: 0.872759997844696

Trial 06 summary
Hyperparameters:
units: 416
learning_rate: 0.001
activation: relu
Score: 0.8708800077438354

Trial 22 summary
Hyperparameters:
units: 288
learning_rate: 0.001
activation: relu
Score: 0.870199978351593

Trial 00 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: sigmoid
Score: 0.8632799983024597

Trial 19 summary
Hyperparameters:
units: 192
learning_rate: 0.001
activation: sigmoid
Score: 0.8575999736785889

Trial 04 summary
Hyperparameters:
units: 96
learning_rate: 0.001
activation: sigmoid
Score: 0.8443999886512756

Trial 07 summary
Hyperparameters:
units: 224
learning_rate: 0.01
activation: sigmoid
Score:

 ### Results

Identify and write the the best performing hyperparameter combination and model score.
Note that because this is Random Search, multiple runs might have slighly different outcomes.



Trial 13 summary
Hyperparameters:
units: 384
learning_rate: 0.001
activation: relu
Score: 0.8747599720954895

------
## 2.3 Bayesian Optimization with `keras-tuner`

![](https://upload.wikimedia.org/wikipedia/commons/0/02/GpParBayesAnimationSmall.gif)

Be sure to check out the [**docs for Keras-Tuner**](https://keras-team.github.io/keras-tuner/documentation/tuners/). Here you can read about the input parameters for the `BayesianOptimization` tuner.

Pay special attention to these `BayesianOptimization` parameters: `num_initial_points` and `beta`.

`num_initial_points`:

Number of randomly selected hyperparameter combinations to try before applying bayesian probability to determine liklihood of which param combo to try next based on expected improvement


`beta`:

Larger values means more willing to explore new hyperparameter combinations (analogous to searching for the global minimum in Gradient Descent), smaller values means that it is less willing to try new hyperparameter combinations (analogous to getting stuck in a local minimum in Gradient Descent).

As a start, error on the side of larger values. What defines a small or large value you ask? That question would pull us into the mathematical intricacies of Bayesian Optimization and Gaussian Processes. For simplicity, notice that the default value is 2.6 and work from there.

In [67]:
# we know that 24 samples is about 25% of 96 possible hyper-parameter combos
# let's set up a run with the same parameters we used for RandomSearch() so the comparison will be apples-to-apples
# feel free to play with any of these numbers later
max_trials=16 # It should be better than 24 at random search but Bayesian is a little smarter
num_initial_points=5 # to start with before applying probability
beta=5.0  # how much do we wanna explore new hyperparameter combos to find the global minimum?

#### Instantiate a `BayesianOptimization()` object for your grid search

In [68]:
bayesian_tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=max_trials,
    hyperparameters=hp, # pass in our hyperparameter dictionary
    num_initial_points=num_initial_points,
    beta=beta,
    seed=1234,
    directory='./keras-tuner-trial',
    project_name='bayesian_optimization_4',
)

In [69]:
bayesian_tuner.search(
    X_train,
    y_train,
    epochs=3,
    validation_data=(X_test, y_test),
)

Trial 16 Complete [00h 01m 00s]
val_accuracy: 0.8356000185012817

Best val_accuracy So Far: 0.8749200105667114
Total elapsed time: 00h 16m 08s


In [70]:
bayesian_tuner.results_summary()

Results summary
Results in ./keras-tuner-trial/bayesian_optimization_4
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 06 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: relu
Score: 0.8749200105667114

Trial 09 summary
Hyperparameters:
units: 448
learning_rate: 0.001
activation: relu
Score: 0.8718799948692322

Trial 00 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: sigmoid
Score: 0.8633599877357483

Trial 07 summary
Hyperparameters:
units: 160
learning_rate: 0.001
activation: relu
Score: 0.8613200187683105

Trial 08 summary
Hyperparameters:
units: 320
learning_rate: 0.001
activation: sigmoid
Score: 0.8595600128173828

Trial 04 summary
Hyperparameters:
units: 96
learning_rate: 0.001
activation: sigmoid
Score: 0.8436800241470337

Trial 14 summary
Hyperparameters:
units: 512
learning_rate: 0.01
activation: sigmoid
Score: 0.8362399935722351

Trial 15 summary
Hyperparameters:
units: 256
learning_rate: 0.01
activation: 

 ### Results

Identify and write the the best performing hyperparameter combination and model score.
Note that because this is  Bayesian Optimization, multiple runs might have slighly different outcomes.



Trial 00 summary
Hyperparameters:
units: 512
learning_rate: 0.001
activation: sigmoid
Score: 0.8633599877357483

We should point out that Gridsearch split the training set internally and created a test set whereas keras-tuner allows us to pass in a test set. This means that the keras-tuner algorithms were using one test set and our skearn GridSearchCV was using a different test set - so this isn't a perfectly exact 1-to-1 comparision but it'll have to do. In order to compensate for this, we did score the best model on the same test set that keras-tuner used.

_______

# Conclusion

The spirit of this experiment is to expose you to the idea of benchmarking and comparing the trade-offs of various gridsearch approaches.

Even if we did find a way to pass in the original test set into GridSearchCV, we can see that both Random Search and Bayesian Optimization are arguably better alternatives to a brute force grid search when we consider the trade-offs of run time and locating the best performing model.

----

# Stretch Goals

- Feel free to run whatever gridsearch experiments on whatever models you like!

In [ ]:
# this is your open playground - be free to explore as you wish
